# Welcome to the BeeAI Framework Workshop 🐝


🎯 Scenario: The Field Marketing Lead has asked you to help prepare their team for conference season. You create a Conference Prep Agent that uses 3 tools: web search to collect relevant news and search for up to date information, Wikipedia to provide company history and details, and the team's internal notes and artifacts.

📚 What You'll Learn

- System Prompts - The foundation of agent behavior
- RequirementAgent - BeeAI's powerful agent implementation that provides control over agent behavior
- LLM Providers - Local and hosted model options
- Memory Systems - Maintaining conversation context
- Tools Integration - Extending agent capabilities
- Conditional Requirements - Enforcing business logic and rules
- Observability - Monitoring and debugging agents

## 🔧 Setup
First, let's install the BeeAI Framework and set up our environment.

- setting up the observability so we can capture and log the actions our agent takes
- getting the "internal documents"

In [ ]:
%pip install -Uqq arize-phoenix s3fs "fsspec==2025.3.0" gcsfs nest_asyncio jedi markdown \
 openinference-instrumentation-beeai "beeai-framework[duckduckgo,llama_index,wikipedia]==0.1.53" \
 langchain langchain_community wikipedia unstructured "requests==2.32.4"

# The following wraps Notebook output
from IPython.display import HTML, display
def set_css(*_, **__):
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))
get_ipython().events.register("pre_run_cell", set_css)

Now let's import the necessary modules:


In [ ]:
import os
import asyncio
import time
import phoenix as px
import ipywidgets
from typing import Any
from datetime import date
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.types import RequirementAgentOutput
from beeai_framework.agents.requirement.requirements import Requirement, Rule
from beeai_framework.agents.requirement.requirements.conditional import ConditionalRequirement
from beeai_framework.backend import ChatModel, ChatModelParameters
from beeai_framework.backend.document_loader import DocumentLoader
from beeai_framework.backend.embedding import EmbeddingModel
from beeai_framework.backend.text_splitter import TextSplitter
from beeai_framework.backend.vector_store import VectorStore
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import GlobalTrajectoryMiddleware
from beeai_framework.tools import StringToolOutput, Tool, tool
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
from beeai_framework.tools.search.retrieval import VectorStoreSearchTool
from beeai_framework.tools.think import ThinkTool
from beeai_framework.tools.weather import OpenMeteoTool
from beeai_framework.tools.types import ToolRunOptions
from openinference.instrumentation.beeai import BeeAIInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

 ## 1️⃣ LLM Providers: Choose Your AI Engine

BeeAI Framework supports 10+ LLM providers including Ollama, Groq, OpenAI, Watsonx.ai, and more, giving you flexibility to choose local or hosted models based on your needs. In this workshop we'll be working Ollama, so you will be running the model locally. You can find the documentation on how to connect to other providers [here](https://framework.beeai.dev/modules/backend).


### *❗* Exercise: Select your Language Model Provider

Change the `provider` and `model` variables to your desired provider and model.

If you select a provider that requires an API key URL or Project_ID, select the key icon on the left menu and set the variables to match those in the userdata.get() function.

Try several models to see how your agent performs. Note that you may need to modify the system prompt for each model, as they all have their own system prompt best practice.

In [ ]:
#Use widgets to show provider choices
providers=ipywidgets.ToggleButtons(options=['ollama','openai','watsonx'])
display(providers)

In Colab, install and start Ollama for providing the Embedding Model.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

In [ ]:
provider=providers.value
from google.colab import userdata
# Ollama - No parameters required
if provider=="ollama":
    model="granite4:tiny-h"
    #model="granite3.3"
    provider_model=provider+":"+model
    !ollama pull $model
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1))
# OpenAI - Place OpenAI API Key in Colab Secrets (key icon) as OPENAI_KEY
elif provider=="openai":
    model="gpt-5-mini"
    provider_model=provider+":"+model
    api_key = userdata.get('OPENAI_KEY')             #Set secret value using key in left menu
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1), api_key=api_key)
# WatsonX - Place Project ID, API Key and WatsonX URL in Colab Secrets (key icon)
elif provider=="watsonx":
    model="ibm/granite-3-8b-instruct"
    provider_model=provider+":"+model
    project_id = userdata.get('WATSONX_PROJECT_ID')  #Set secret value using key in left menu
    api_key = userdata.get('WATSONX_APIKEY')         #Set secret value using key in left menu
    base_url = userdata.get('WATSONX_URL')           #Set secret value using key in left menu
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1), project_id=project_id, api_key=api_key, base_url=base_url)
else:
  print("Provider " + provider + " undefined")

## 2️⃣ Understanding System Prompts: The Agent's Foundation

What is a System Prompt?
A system prompt is the foundational instruction that defines your agent's identity, role, and behavior. Think of it as the agent's "job description" and "training manual" rolled into one. Each model responds differently to the same system prompt, so experimentation is necessary.

Some key components of a strong system prompt:

- Identity: Who is the agent?
- Role: What is their function?
- Context: What environment are they operating in?
- Rules: What constraints and guidelines must they follow?
- Knowledge: What domain-specific information do they need?

### *❗* Exercise: Customize Your System Prompt
Try modifying the system prompt. Customize the "basic rules" section to add your own. Note that changes to the system prompt will affect the performance of the model. Creating a great `System Prompt` is an art, not a science.

In [ ]:
todays_date = date.today().strftime("%B %d, %Y")
instruct_prompt = f"""You help field marketing teams prep for conferences by answering questions on companies that they need to prepare to talk to. You produce quick and actionable briefs, doing your best to anwer the user's question.

Today's date is {todays_date}.

Tools:
- ThinkTool: Helps you plan and reason before you act. Use this tool when you need to think.
- DuckDuckGoSearchTool: Use this tool to collect current information on agendas, speakers, news, competitor moves. Include title + date + link to the resources you find in your answer. Do not use this tool for internal notes or artifacts.
- wikipedia_tool: Use this tool to get company/org history (not for breaking news). Only look up company names as the input.
- internal_document_search: past meetings, playbooks, artifacts. If you use information from this in yoour response, label it as as [Internal]. Always use this tool when internal notes or content is references.

Basic Rules:
- Be concise and practical.
- Favor recent info (agenda/news ≤30 days; exec changes/funding ≤180 days); flag older items.
- If you don't know, say so. Don't make things up.
"""

## 3️⃣ Memory Systems: Maintaining Context across iterations or sessions
Why Memory Matters
Short term memory allows agents to:
- Remember previous conversation turns
- Build on past interactions
- Maintain context across multiple queries

Long Term memory allows agents to:
- Learn from user preferences
- Provided a grounded source of truth
- Pull data from non public data sources

### *❗* Exercise: Choose your memory strategy
`Memory` is an important piece of AI agents. Experiment with the different startegies by running only 1 of the cells and finishing the notebook. Optionally, once you have ran the entire notebook, come back and select a differnt memory approach and see how that affects the agent output.

**BeeAI Memory Types**

BeeAI Framework provides four memory strategies optimized for different use cases, with support for custom memory:

1. SlidingWindowMemory
- Use case: Production applications with long conversations
- Behavior: Maintains a fixed number of recent messages
- Pros: Controlled memory usage, predictable costs
- Cons: May lose important early context

In [ ]:
from beeai_framework.memory import SlidingMemory, SlidingMemoryConfig

memory = SlidingMemory(SlidingMemoryConfig(
            size=3,
            handlers={"removal_selector": lambda messages: messages[0]} # Remove the oldest message
            ))

2. TokenWindowMemory
Use case: Cost-sensitive applications
Behavior: Maintains messages within token budget
Pros: Cost control, adapts to message length
Cons: Complex to predict exact retention

In [ ]:
from beeai_framework.memory import TokenMemory
import math

memory = TokenMemory(
    llm=llm,
    max_tokens=None,  # Will be inferred from LLM
    capacity_threshold=0.75,
    sync_threshold=0.25,
    handlers={
        "removal_selector": lambda messages: next((msg for msg in messages if msg.role != Role.SYSTEM), messages[0]),
        "estimate": lambda msg: math.ceil((len(msg.role) + len(msg.text)) / 4),
    },
)

3. SummarizeMemory
- Use case: Long-running agents that need historical context
- Behavior: Summarizes old messages to maintain key information
- Pros: Retains important information efficiently
- Cons: May lose nuanced details in summaries

In [ ]:
from beeai_framework.memory import SummarizeMemory

memory = SummarizeMemory(llm)

4. UnconstrainedMemory (Default)

- Useful for: Development, testing, simple applications
- Behavior: Stores all messages indefinitely
- Pros: Simple, maintains full context
- Cons: Can become expensive with long conversations

In [ ]:
from beeai_framework.memory import UnconstrainedMemory

memory = UnconstrainedMemory()

### *❗* Exercise: Memory Comparison
Chose the memory type you want your agent to have. You can also go back and compare your choice of memory types and how it affected the agent response.

## 4️⃣ Tools: Enabling LLMs to Take Action

What Are Tools?
Tools are external capabilities that extend your agent beyond just generating text. They can be API calls, code, or even calls to other AI models. They can allow agents to:

- Access real-time data (internet search, API calls to live data)
- Perform calculations (using code generation tools)
- Interact with APIs (databases, web services)
- Process files (call functions that read, modify, or write files)
- Interact with `MCP Servers`

The BeeAI framework provides [built in tools](https://framework.beeai.dev/modules/tools#built-in-tools) for common tool types, but also provides the ability to create [custom tools](https://framework.beeai.dev/modules/tools#creating-custom-tools).

### Adding Framework Provided Tools

The **Think tool** encourages a Re-Act pattern where the agent reasons and plans before calling a tool.


In [ ]:
from beeai_framework.tools.think import ThinkTool

think_tool = ThinkTool()

The **DuckDuckGoSearchTool** is a Web Search tool that provides relevant data from the internet to the LLM


In [ ]:
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()

### Adding Custom Tools


There are 2 ways to provide custom tools to your agent. For simple tools you can use the `@tool` decorator above the function. For more complex tools, you can extend the `Tool Class` and customize things such as the run time and tool execution.  

We will create a simple custom tool with the `@tool` decorator. Our tool must have a doc string, so that the agent understands when and how it should use that tool. The tool name will defualt to the function name.

To learn more about advanced tool customization, take a look at this section in the [documentation](https://framework.beeai.dev/modules/tools#advanced-custom-tool).

In [ ]:
from beeai_framework.tools.search.wikipedia import WikipediaTool, WikipediaToolInput

@tool
async def wikipedia_tool(query: str) -> str:
  """
  Search factual and historical information, including biography, history, politics, geography, society, culture,
  science, technology, people, animal species, mathematics, and other subjects.

  Args:
      query: The topic or question to search for on Wikipedia.

  Returns:
      The information found via searching Wikipedia.
  """
  full_text = False
  language = "en"
  tool = WikipediaTool(language=language)
  response = await tool.run(WikipediaToolInput(query=query, full_text=False))
  return response.get_text_content()

## 5️⃣ Creating a RAG (Retrieval Augmented Generation) Tool to Search Internal Documents

`RAG` (Retrieval-Augmented Generation) is “search + write”: you ask a question, the system retrieves the most relevant snippets from an indexed knowledge base (via embeddings) and the model composes an answer grounded in those snippets.

**We created synthetic (made-up) documents to simulate a company knowledge base:**
- Security checklists
- Call notes
- Artifacts

We made sets of these for Spotify, Siemens, and Moderna.

**Important:** This is demonstration-only data and does not reflect real information about those companies.

The BeeAI Framework has built in abstractions to make RAG simple to implement. Read more about it [here](https://framework.beeai.dev/modules/rag).

First, we must pull an embedding model which converts text into numerical vectors so we can compare meanings and retrieve the most relevant snippets. The original document is:
1. preprocessed (cleaned + broken into chunks)
2. ran through the embedding algorithm
3. stored in the vector database


In [ ]:
!ollama pull nomic-embed-text:latest
embedding_model = EmbeddingModel.from_name("ollama:nomic-embed-text")

### *❗* Exercise: Internal documents
Take a look at the internal documents so you know what type of questions to ask your agent


In [ ]:
# Read the synthetic data from the public github repo
!curl --output rag_conference_prep_agent.txt https://raw.githubusercontent.com/IBM/beeai-workshop/refs/heads/main/intro_beeai_framework/rag_conference_prep_agent.txt

with open('rag_conference_prep_agent.txt', 'r') as file:
    content = file.read()
print(content)

Load the document using the `DocumentLoader` and split the document into chunks using the `text_splitter`.

In [ ]:
loader = DocumentLoader.from_name(
    name="langchain:UnstructuredMarkdownLoader", file_path="rag_conference_prep_agent.txt"
)
try:
    documents = await loader.load()
except Exception as e:
    print(f"Failed to load documents: {e}")

# Split documents into chunks
text_splitter = TextSplitter.from_name(
    name="langchain:RecursiveCharacterTextSplitter", chunk_size=1000, chunk_overlap=200
)
try:
    documents = await text_splitter.split_documents(documents)
except Exception as e:
    print(f"Failed to split documents: {e}")
print(f"Loaded {len(documents)} document chunks")

Create the `TemporalVectorStore`, which means this vector store also tracks time.

In [ ]:
# Create vector store and add documents
vector_store = VectorStore.from_name(name="beeai:TemporalVectorStore", embedding_model=embedding_model)
await vector_store.add_documents(documents=documents)
print("Vector store populated with documents")

Create the `internal_document_search` tool! Because the `VectorStoreSearchTool` is a built in tool wrapper, we don't need to use the `@tool` decorator or extend the custom `Tool class`.

In [ ]:
# Create the vector store search tool
internal_document_search = VectorStoreSearchTool(vector_store=vector_store)

## 6️⃣ Conditional Requirements: Guiding Agent Behavior


What Are Conditional Requirements?
[Conditional requirements](https://framework.beeai.dev/experimental/requirement-agent#conditional-requirement) ensure your agents are reliable by controlling when and how tools are used. They're like business rules for agent behavior. You can make them as strict (esentially writing a static workflow) or flexible (no rules! LLM decides) as you'd like.

The rules that you enforce may seem simple in the BeeAI framework, but in other frameworks they require ~5X the amount of code. Check out this [blog](https://beeai.dev/blog/reliable-ai-agents) where we built the same agent in BeeAI and other agent framework LangGraph.

These conditional requirements enforce the following in only 3 lines of code:
1. The agent must call the think tool as the first tool call. It is not allowed to call it consecutive times in a row.
2. The wikipedia_tool can only be called after the think tool, but not consecutively. It has a relative priority of 10.
3. The DuckDuckGo Internet search tool can also only be called after the Think tool, it is allowed to be called up to 3 times, it must be invoked at least once, and it has a relative priority of 15.
4. The internal_document_search tool can only be called after the think tool, it is allowed to be called multiple times in a row, it must be called at least once, and it has a relative priority of 20.



In [ ]:
requirement_1 = ConditionalRequirement(ThinkTool, consecutive_allowed=False, force_at_step=1 )

requirement_2 = ConditionalRequirement(wikipedia_tool, only_after=ThinkTool, consecutive_allowed=True, priority=10,)

requirement_3 = ConditionalRequirement(DuckDuckGoSearchTool, only_after=ThinkTool, consecutive_allowed=True, min_invocations=1, max_invocations=3 ,priority=15,)

requirement_4 = ConditionalRequirement(internal_document_search, only_after=ThinkTool, consecutive_allowed=True, min_invocations=1, priority=20,)

## Explore Observability: See what is happening under the hood

Create the function that sets up observability using `OpenTelemetry` and [Arize's Phoenix Platform](https://arize.com/docs/phoenix/inferences/how-to-inferences/manage-the-app). There a several ways to view what is happening under the hood of your agent. View the observability documentation [here](https://framework.beeai.dev/modules/observability).

In [ ]:
def setup_observability(endpoint: str = "http://localhost:6006/v1/traces") -> None:
    """
    Sets up OpenTelemetry with OTLP HTTP exporter and instruments the beeai framework.
    """
    resource = Resource(attributes={})
    tracer_provider = trace_sdk.TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
    trace_api.set_tracer_provider(tracer_provider)

    BeeAIInstrumentor().instrument()

In [ ]:
load_dotenv()
# Enable OpenTelemetry integration
setup_observability("http://localhost:6006/v1/traces")
px_session = px.launch_app()

##  7️⃣ Assemble Your Reliable BeeAI Agent

This is the part we've been working towards! Let's assemble the agent with all the parts we created.

In [ ]:
agent = RequirementAgent(
    llm=llm,
    instructions= instruct_prompt,
    memory = memory,
    tools=[ThinkTool(), DuckDuckGoSearchTool(), wikipedia_tool, internal_document_search],
    requirements=[
      requirement_1,
      requirement_2,
      requirement_3,
      requirement_4
    ],
    # Log intermediate steps to the console
    middlewares=[GlobalTrajectoryMiddleware(included=[Tool])],
)

### *❗* Exercise: Test Your Agent
Remember that your agent is meant to prep the field marketing team for upcoming conferences and has a limited set of "internal documents". Make up your own question or ask one of the sample ones below!


**Sample Questions:**
- Brief me for a Shopify meeting at the conference. Give me an overview of the company, some recent news about them, and anything important I need to know from our internal notes.

- I'm planning on meeting the Moderna rep at the next conference. Give me a one pager and remind me where we left off on previous discussions.

- Build a security talking sheet for Siemens Energy. How does their strategy compare to their competitors'?

In [ ]:
question = "I'm planning on meeting the Moderna rep at the next conference. Give me a one pager and remind me where we left off on previous internal discussions."

Run the agent with specific execution settings.

### *❗* Exercise: Test Your Agent
Change the execution settings and see what happens. Does your agent run out of iterations? Every task is different and its important to balance flexibility with control.

In [ ]:
response = await agent.run(question, max_retries_per_step=3, total_max_retries=25)
print(response.last_message.text)

In [ ]:
px_session.view()